### Note: run following cell to install earth engine reqs.

In [ ]:
pip install earthengine-api --upgrade

#### Further note: need to create api key with google account and follow the authentication workflow

In [ ]:
import ee
ee.Authenticate()

#### Experimental code
For comparing different publicly available satellite imagery sources.

In [ ]:
import ee

ee.Initialize()

def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
        .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000)

collection_type = 'COPERNICUS' # 'COPERNICUS' , 'NAIP'

# Define the image collection
if collection_type == 'LANDSAT':
    collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')
elif collection_type == 'COPERNICUS':
    collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
elif collection_type == 'NAIP':
    collection = ee.ImageCollection('USDA/NAIP/DOQQ')

# Define the region of interest
point = ee.Geometry.Point([-122.262, 37.8719])

# Buffer the point and create a square region around it (5000m x 5000m)
region = point.buffer(1000).bounds()

# Filter the collection for a specific date
filtered = collection.filterDate('2019-01-01', '2022-01-31')

# filtered = filtered.map(maskS2clouds)

# Filter the collection by region
filtered = filtered.filterBounds(region)

# Check if the filtered collection is empty
if filtered.size().getInfo() == 0:
    raise ValueError("No images found in the specified date range.")

# Sort the collection by metadata
if collection_type == 'LANDSAT':
    sorted = filtered.sort('CLOUD_COVER')
else:
    sorted = filtered.sort('system:time_start', False) # Most recent images first

# Get the first image in the sorted collection
image = ee.Image(sorted.first())

# Check if the image is None
if image is None:
    raise ValueError("No images found after sorting.")


# print the date of the image
date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
print('Date of the image:', date.getInfo())

# print the cloud cover of the image
if collection_type == 'LANDSAT':
    print('Cloud cover (%):', image.get('CLOUD_COVER').getInfo())

# Get a URL for the image (also specify region and scale if necessary)
if collection_type == 'LANDSAT':
    # Define the visualization parameters
    visParams = {
      'bands': ['B4', 'B3', 'B2'],
      'min': 0,
      'max': 0.3,
      'gamma': [1.0, 1.0, 1.0]
    }

    url = image.getThumbUrl({
        'region': region.getInfo(),
        'scale': 30,  # Adjust the scale as needed. Landsat images have 30m resolution.
        **visParams
    })
elif collection_type == 'COPERNICUS':
    # Define the visualization parameters
    visParams = {
      'bands': ['B4', 'B3', 'B2'],
      'min': 0.0,
      'max': 0.3,
    }
    url = image.getThumbUrl({
        'region': region.getInfo(),
        'scale': 10,  # Adjust the scale as needed. Sentinel-2 images have 10m resolution.
        **visParams
    })
elif collection_type == 'NAIP':
    # Define the visualization parameters
    visParams = {
      'bands': ['R', 'G', 'B'],
      'min': 0,
      'max': 255,
    }
    url = image.getThumbUrl({
        'region': region.getInfo(),
        'scale': 1,  # Adjust the scale as needed. NAIP images have 1m resolution.
        **visParams
    })

print(url)